In [1]:
import argparse
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import requests
import re

In [2]:
# Replace 'YOUR_API_KEY' with your actual API key.
DEVELOPER_KEY = 'GOOGLE DEVELOPER KEY'
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

In [6]:
import os
import re

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
# from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.document_loaders import PyPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import HumanMessage
import nest_asyncio
nest_asyncio.apply()

In [7]:
os.environ['GOOGLE_API_KEY'] = "YOUR API KEY"
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro-latest",
    safety_settings={
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)
embed_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [8]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

In [9]:
def get_captions(video_id: str) -> str:
    """Fetch the caption ID for a given YouTube video ID if English captions are available."""
    captions_url = f"https://www.googleapis.com/youtube/v3/captions?part=snippet&videoId={video_id}&key={DEVELOPER_KEY}"
    response = requests.get(captions_url)
    if response.status_code == 200:
        captions = response.json().get('items', [])
        for caption in captions:
            if caption['snippet']['language'] == 'en':
                return caption['id']
    return None

def download_captions(caption_id: str) -> str:
    """Download and return the captions for a given caption ID."""
    download_url = f"https://www.googleapis.com/youtube/v3/captions/{caption_id}?tfmt=ttml&key={DEVELOPER_KEY}"
    response = requests.get(download_url)
    if response.status_code == 200:
        ttml_data = response.text
        captions = re.findall(r'<p.*?>(.*?)</p>', ttml_data)
        return " ".join(captions)
    return "No captions available."

In [18]:
@tool
def youtube_search(query: str, max_results: int = 10) -> str:
    """
    Search YouTube for videos matching the query related to dental health and dental procedures, as requested by DentalCare.AI users. Provide a curated list of high-quality video resources relevant to dental topics.

    Args:
        query (str): The dental-related search term to use.
        max_results (int): The maximum number of results to return (default is 10).

    Returns:
        str: A formatted string containing the video titles, URLs, descriptions, and captions (if available) for the top results.
        
    Functionality:
        - Search YouTube for videos on topics such as dental procedures, oral health tips, emergency dental advice, post-procedure care, and dental education.
        - Ensure videos are from reputable sources and cover the requested topics in detail.
        - Provide relevant information in a user-friendly format for DentalCare.AI users seeking video resources.
    """
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)
    
    search_response = youtube.search().list(
        q=query,
        part='id,snippet',
        maxResults=max_results
    ).execute()

    videos = []
    for item in search_response.get('items', []):
        if item['id']['kind'] == 'youtube#video':
            title = item['snippet']['title']
            video_id = item['id']['videoId']
            description = item['snippet']['description']
            caption_id = get_captions(video_id)
            captions = download_captions(caption_id) if caption_id else "No captions available."
            videos.append(f"Title: {title}\nURL: https://www.youtube.com/watch?v={video_id}\nDescription: {description}\nCaptions: {captions}\n")
    
    return "\n\n".join(videos)


In [19]:

from langgraph.prebuilt import create_react_agent
tools = [youtube_search]
memory = SqliteSaver.from_conn_string(":memory:")
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

In [20]:
from langchain_core.messages import HumanMessage
config = {"configurable": {"thread_id": "abc123"}}

for s in agent_executor.stream(
    {"messages": [HumanMessage(content="root canal surgery")]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'youtube_search', 'arguments': '{"query": "root canal surgery"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-0495158d-a2d6-4fbf-957a-57c549318abb-0', tool_calls=[{'name': 'youtube_search', 'args': {'query': 'root canal surgery'}, 'id': '656c60ee-9d15-44bd-92f1-1afce24a5964', 'type': 'tool_call'}], usage_metadata={'input_tokens': 237, 'output_tokens': 17, 'total_tokens': 254})]}}
----
{'tools': {'messages': [ToolMessage(content="Title: Deliv

**Complete Response in MD below**

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'youtube_search', 'arguments': '{"query": "root canal surgery"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-0495158d-a2d6-4fbf-957a-57c549318abb-0', tool_calls=[{'name': 'youtube_search', 'args': {'query': 'root canal surgery'}, 'id': '656c60ee-9d15-44bd-92f1-1afce24a5964', 'type': 'tool_call'}], usage_metadata={'input_tokens': 237, 'output_tokens': 17, 'total_tokens': 254})]}}
----
{'tools': {'messages': [ToolMessage(content="Title: Delivering MTApex™ bioceramic root canal sealer\nURL: https://www.youtube.com/watch?v=CPjbmHIhirc\nDescription: MTApex™ bioceramic root canal sealer is deliverable via a NaviTip™ 29 ga single sideport tip or gutta percha. Learn more at ...\nCaptions: No captions available.\n\n\nTitle: Root Canal Procedure Step by Step\nURL: https://www.youtube.com/watch?v=81qSdFYKRcc\nDescription: Let's see what happens during a root canal procedure! A root canal procedure is extremely safe and one of the best ways to ...\nCaptions: No captions available.\n\n\nTitle: Root Canal Treatment\nURL: https://www.youtube.com/watch?v=wZ1MtdkkZXg\nDescription: This 3D medical animation features a procedure called 'Root canal treatment'. This is a sequence of steps to treat the pulp of a ...\nCaptions: No captions available.\n\n\nTitle: ROOT CANAL treatment step by step  - 3D video of endodontics for tooth decay ©\nURL: https://www.youtube.com/watch?v=UQH3GWsCmr8\nDescription: How is a root canal treatment done? When is it necessary? 3D video of Clínica Médico Dental Pardiñas (http://j.mp/cPardinas) ...\nCaptions: No captions available.\n\n\nTitle: Root Canal Treatment RCT (Step by Step Animation)\nURL: https://www.youtube.com/watch?v=0s35QCFg7p0\nDescription: Root canal more popularly know as RCT is a very simple procedure done under local anaesthesia. Modern day root canal is ...\nCaptions: No captions available.\n\n\nTitle: Root Canal Surgery\nURL: https://www.youtube.com/watch?v=z74yyNPkjyA\nDescription: Visit us at http://www.surgerysquad.com/ What could be worse than a root canal? A lot of things! The root canal is probably one of ...\nCaptions: No captions available.\n\n\nTitle: What&#39;s Really Happening During A Root Canal Procedure\nURL: https://www.youtube.com/watch?v=oBGqPBDg3tw\nDescription: Root canal therapy (aka a root canal) is the process of repairing and saving a damaged or infected tooth. This video demonstrates ...\nCaptions: No captions available.\n\n\nTitle: Why would someone need a Root Canal? 🦷💉#trulydental #dentist #qanda #rootcanal #causes #explained\nURL: https://www.youtube.com/watch?v=lOiSaYL-hFs\nDescription: \nCaptions: No captions available.\n\n\nTitle: LIVE: Root Canal Procedure under the Microscope w/ Specialist\nURL: https://www.youtube.com/watch?v=OIoZ9EehR0I\nDescription: Full Length Video - https://youtu.be/d4mozZSmuF4 FREE ENDO GUIDE & NEWSLETTER ...\nCaptions: No captions available.\n\n\nTitle: Endodontists Put Root Canal Patients at Ease\nURL: https://www.youtube.com/watch?v=HSsA6qnqlNQ\nDescription: Features interviews with root canal patients describing their positive, pain-free experiences, while endodontists explain what a ...\nCaptions: No captions available.\n", name='youtube_search', tool_call_id='656c60ee-9d15-44bd-92f1-1afce24a5964')]}}
----
{'agent': {'messages': [AIMessage(content="Title: Delivering MTApex™ bioceramic root canal sealer\nURL: https://www.youtube.com/watch?v=CPjbmHIhirc\nDescription: MTApex™ bioceramic root canal sealer is deliverable via a NaviTip™ 29 ga single sideport tip or gutta percha. Learn more at ...\nCaptions: No captions available.\n\n\nTitle: Root Canal Procedure Step by Step\nURL: https://www.youtube.com/watch?v=81qSdFYKRcc\nDescription: Let's see what happens during a root canal procedure! A root canal procedure is extremely safe and one of the best ways to ...\nCaptions: No captions available.\n\n\nTitle: Root Canal Treatment\nURL: https://www.youtube.com/watch?v=wZ1MtdkkZXg\nDescription: This 3D medical animation features a procedure called 'Root canal treatment'. This is a sequence of steps to treat the pulp of a ...\nCaptions: No captions available.\n\n\nTitle: ROOT CANAL treatment step by step  - 3D video of endodontics for tooth decay 👀\nURL: https://www.youtube.com/watch?v=UQH3GWsCmr8\nDescription: How is a root canal treatment done? When is it necessary? 3D video of Clínica Médico Dental Pardiñas (http://j.mp/cPardinas) ...\nCaptions: No captions available.\n\n\nTitle: Root Canal Treatment RCT (Step by Step Animation)\nURL: https://www.youtube.com/watch?v=0s35QCFg7p0\nDescription: Root canal more popularly know as RCT is a very simple procedure done under local anaesthesia. Modern day root canal is ...\nCaptions: No captions available.\n\n\nTitle: Root Canal Surgery\nURL: https://www.youtube.com/watch?v=z74yyNPkjyA\nDescription: Visit us at http://www.surgerysquad.com/ What could be worse than a root canal? A lot of things! The root canal is probably one of ...\nCaptions: No captions available.\n\n\nTitle: What's Really Happening During A Root Canal Procedure\nURL: https://www.youtube.com/watch?v=oBGqPBDg3tw\nDescription: Root canal therapy (aka a root canal) is the process of repairing and saving a damaged or infected tooth. This video demonstrates ...\nCaptions: No captions available.\n\n\nTitle: Why would someone need a Root Canal? 🤔🦷#trulydental #dentist #qanda #rootcanal #causes #explained\nURL: https://www.youtube.com/watch?v=lOiSaYL-hFs\nDescription: \nCaptions: No captions available.\n\n\nTitle: LIVE: Root Canal Procedure under the Microscope w/ Specialist\nURL: https://www.youtube.com/watch?v=OIoZ9EehR0I\nDescription: Full Length Video - https://youtu.be/d4mozZSmuF4 FREE ENDO GUIDE & NEWSLETTER ...\nCaptions: No captions available.\n\n\nTitle: Endodontists Put Root Canal Patients at Ease\nURL: https://www.youtube.com/watch?v=HSsA6qnqlNQ\nDescription: Features interviews with root canal patients describing their positive, pain-free experiences, while endodontists explain what a ...\nCaptions: No captions available.", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-83a3b3e7-06ef-4dd2-b642-1e364ded9f35-0', usage_metadata={'input_tokens': 1167, 'output_tokens': 732, 'total_tokens': 1899})]}}
----
